In [102]:
#01_DifferentialAbundance_and_cohens_D_adjust_effect
#Derivated from: 01_DifferentialAbundance_and_cohens_D
#
#
#Purpose: Use linear model to identify differentially abundant features (for each omics)
#[1] features (or predictors) that have significant coefficients (p < 0.05)
#           -> whether ACPA status (predictor) has affected the abundance of the feature (response).
#[2] features Cohend's D above medium (i.e., 0.5)
#features that fulfills [1] + [2] will considerexd as differentially abundant.

library("effsize")
library(lme4)
library(lmerTest)
library(stringr)
library(effects)
library(dplyr)


In [133]:
main <- function(input_data_file, patient_info_file, output_dir, output_str){
    
    input_data_df <- read.csv(input_data_file, sep="\t", header=TRUE, row.names=1)
    patient_info_df <- read.csv(patient_info_file, sep="\t", header=TRUE, row.names=1)

    merged_data_df <- rbind(patient_info_df[1:nrow(patient_info_df),], input_data_df) #acpa    
    merged_data_df <- as.data.frame(t(merged_data_df))

    #acpa-neg: 2, acpa-pos: 1, control: 0
    #control vs acpa neg
    control_vs_acpa_neg <- filter(merged_data_df, acpa == 0 | acpa == 2)
    control_vs_acpa_neg$acpa[control_vs_acpa_neg$acpa == 2] <- 1   #changing class 2 -> 1; making 0 vs 1

    #control vs acpa pos
    control_vs_acpa_pos <- filter(merged_data_df, acpa == 0 | acpa == 1)

    #acpa neg vs acpa pos
    acpa_neg_vs_acpa_pos <- filter(merged_data_df, acpa == 1 | acpa == 2)
    acpa_neg_vs_acpa_pos$acpa[acpa_neg_vs_acpa_pos$acpa == 1] <- 0   #changing class 1 -> 0
    acpa_neg_vs_acpa_pos$acpa[acpa_neg_vs_acpa_pos$acpa == 2] <- 1   #changing class 2 -> 1
   
    run_cohenD_and_glm(control_vs_acpa_neg, 1, 0, output_dir, output_str, '.cVSneg')
    run_cohenD_and_glm(control_vs_acpa_pos, 1, 0, output_dir, output_str, '.cVSpos')
    run_cohenD_and_glm(acpa_neg_vs_acpa_pos, 0, 1, output_dir, output_str, '.negVSpos')
    
}

run_cohenD_and_glm <- function(data_df, condition_a_num, condition_b_num, output_dir, output_str, output_type){
    
    NUM_FEATURES <- ncol(data_df)
    temp_condition_a_df <- filter(data_df, acpa == condition_a_num) #this is necessary for cohends D
    temp_condition_b_df <- filter(data_df, acpa == condition_b_num) #this is necessary for cohends D   
    
    output_txt <- paste(output_dir, output_str, output_type, '.tsv', sep="") 
    if (file.exists(output_txt)) {
        #Delete file if it exists
        file.remove(output_txt)
    }

    output_string <- "\tcohenD\tfc_case_control\tcoeff\tpval\tdrug_adj_pval\tpred_pval\tbdmard_pval\tcsdmard_pval\tdrug_effect\n"
    cat(output_string, file=output_txt, append=TRUE)
    
    for (i in 1:NUM_FEATURES){
        if (i > 19){
            feature <- colnames(data_df)[i]
            #calculate cohens D between two population
            condition_a_list <- temp_condition_a_df[,i]
            condition_b_list <- temp_condition_b_df[,i]
            
            cohend = cohen.d(condition_a_list,condition_b_list)
            cohend_value <- cohend$estimate
            
            log2fc <- log2(mean(condition_a_list) / mean(condition_b_list))
            
            glm_results <- glm(data_df[,i] ~ acpa, data = data_df) #ACPA margnial model
            feature_coef <- (coef(summary(glm_results))[,1][2])
            feature_pval <- (coef(summary(glm_results))[,4][2])
            
            #calculate the significance of the linear model
            glm_results <- glm(data_df[,i] ~ pred, data = data_df) #pred margnial model
            pred_pval <- (coef(summary(glm_results))[,4][2])

            glm_results <- glm(data_df[,i] ~ bdmard, data = data_df) #bdmard margnial model
            bdmard_pval <- (coef(summary(glm_results))[,4][2])

            glm_results <- glm(data_df[,i] ~ all_csdmard, data = data_df) #csdmard margnial model
            all_csdmard_pval <- (coef(summary(glm_results))[,4][2])

            if (pred_pval < 0.05 || bdmard_pval < 0.05 || all_csdmard_pval < 0.05) {
                drug_eff <- 1
            } else {
                drug_eff <- 0
            }
 
            if (pred_pval < 0.05 && bdmard_pval < 0.05 && all_csdmard_pval < 0.05) {
                #feature is effected by pred, bdmard, all csdmards
                glm_results <- glm(data_df[,i] ~ acpa + pred + bdmard + all_csdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (pred_pval < 0.05 && bdmard_pval < 0.05) {
                #feature is effected by pred, bdmard
                glm_results <- glm(data_df[,i] ~ acpa + pred + bdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (pred_pval < 0.05 && all_csdmard_pval < 0.05) {
                #feature is effected by pred, all csdmards
                glm_results <- glm(data_df[,i] ~ acpa + pred + all_csdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (pred_pval < 0.05) {
                #feature is effected by pred
                glm_results <- glm(data_df[,i] ~ acpa + pred, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (bdmard_pval < 0.05 && all_csdmard_pval < 0.05) {
                #feature is effected by bdmard, all csdmards
                glm_results <- glm(data_df[,i] ~ acpa + bdmard + all_csdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (bdmard_pval < 0.05) {
                #feature is effected by bdmard
                glm_results <- glm(data_df[,i] ~ acpa + bdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } else if (all_csdmard_pval < 0.05) {
                #feature is effected by csdmard
                glm_results <- glm(data_df[,i] ~ acpa + all_csdmard, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            } 
            else {#feature is note effected by drug
                glm_results <- glm(data_df[,i] ~ acpa, data = data_df)
                adj_feature_pval <- (coef(summary(glm_results))[,4][2])
                
            }
            output_string <- paste(feature, "\t", cohend_value, "\t", log2fc, "\t",feature_coef, "\t", feature_pval, "\t", adj_feature_pval, "\t", pred_pval, "\t", bdmard_pval, "\t", all_csdmard_pval, "\t", drug_eff, "\n", sep="")
            cat(output_string, file=output_txt,append=TRUE)
        } 
    }
}


In [134]:
#Main
output_dir = '../../../analysis/statistics/linear_model/differential_abundance/'

#make directory if it does not exist
if (!dir.exists(output_dir)){
dir.create(output_dir)
} else {
    print("Dir already exists!")
}

autoantibody_data_file = '../../../preprocessed_data/autoantibody/sengenics_qnorm_data.v2.tsv'
proteomics_data_file = '../../../preprocessed_data/proteomics/somascan_anml.T.v2.tsv'
metabolomics_data_file = '../../../preprocessed_data/metabolomics/metabolone_raw_norm_preprocessed.v2.tsv'
patient_info_file = '../../../preprocessed_data/meta/patient_info_for_statistics.v3.T.tsv'

[1] "Dir already exists!"


In [135]:
main(autoantibody_data_file, patient_info_file, output_dir, 'autoantibody')
main(metabolomics_data_file, patient_info_file, output_dir, 'metabolomics')
main(proteomics_data_file, patient_info_file, output_dir, 'proteomics')
